In [30]:
import numpy as np
import pandas
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

from mne_connectivity import vector_auto_regression

import seaborn as sns

# Generate Data

In [21]:
rng = np.random.RandomState(0)
n_epochs, n_signals, n_times = 2, 4, 64
data = rng.randn(n_epochs, n_signals, n_times)
times = np.arange(n_times)

print(data.shape)

(2, 4, 64)


In [22]:
# statsmodels only takes in data that is T x K,
# where T is number of time points and K is the number of signals
data = rng.randn(n_times, n_signals)

In [17]:
mdata = sm.datasets.macrodata.load_pandas().data

# prepare the dates index
dates = mdata[['year', 'quarter']].astype(int).astype(str)

quarterly = dates["year"] + "Q" + dates["quarter"]

from statsmodels.tsa.base.datetools import dates_from_str

quarterly = dates_from_str(quarterly)

mdata = mdata[['realgdp','realcons','realinv']]

mdata.index = pandas.DatetimeIndex(quarterly)

data = np.log(mdata).diff().dropna()

In [25]:
print(data.shape)
print()

(64, 4)



# Apply MNE-Conn models

In [6]:
conn = vector_auto_regression(data, times=times, n_jobs=-1)


100%|██████████| 2/2 [00:00<00:00, 137.71it/s]


In [10]:
print(conn)

<EpochConnectivity | , nave : 2, nodes, n_estimated : 4, 16, ~4 kB>


# Apply statsmodels VAR

In [23]:
model = VAR(data)

In [24]:
results = model.fit(1)

In [28]:
coef = results.coefs
print(coef.shape)

sns.heatmap(coef.squeeze(), cmap='viridis')

(1, 4, 4)


In [27]:
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 04, Aug, 2021
Time:                     17:10:53
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   0.761652
Nobs:                     63.0000    HQIC:                  0.348881
Log likelihood:          -340.133    FPE:                    1.08614
AIC:                    0.0812921    Det(Omega_mle):        0.800226
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const        -0.410343         0.126429           -3.246           0.001
L1.y1        -0.150704         0.125845           -1.198           0.231
L1.y2         0.080613         0.111708            0.722           0.471
L1.y3        -0.107395